In [ ]:
from keras.applications import 

In [2]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


# DATA_DIR = 'data'
TRAIN_DIR = "/home/jeniffer/abs/blurDetection/train"
VALID_DIR = "/home/jeniffer/abs/blurDetection/test"
SIZE = (224, 224)
BATCH_SIZE = 64

# Spatial Rich Models kernel initializer function
def srm(shape, dtype=None):
    data = np.load('/home/archimedes/abs/saravanan-rnd/forgery/custom/SRM_Kernels.npy')
    data = np.concatenate((data,data,data), axis = 2)
    return data

class CustomInitializer:
    def __call__(self, shape, dtype=None):
        return srm(shape, dtype=dtype)
    

            
            
if __name__ == "__main__":
    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

    num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
    num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

    gen = keras.preprocessing.image.ImageDataGenerator()
    val_gen = keras.preprocessing.image.ImageDataGenerator()

    batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    
    model = keras.applications.resnet50.ResNet50()

    classes = list(iter(batches.class_indices))
    model.layers.pop()
    print(model.summary())
    for layer in model.layers:
        layer.trainable=True
    last = model.layers[-1].output
    x = Dense(len(classes), activation="softmax")(last)
    finetuned_model = Model(model.input, x)
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    for c in batches.class_indices:
        classes[batches.class_indices[c]] = c
    finetuned_model.classes = classes

    early_stopping = EarlyStopping(patience=10)
    checkpointer = ModelCheckpoint('models/resnet50_blur_weights_{epoch:02d}.h5', verbose=1, save_best_only=True)

    finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=20, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)
    #finetuned_model.save('resnet50_blur.h5')

Found 48000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_______________________

Epoch 1/20
750/750 [==============================] - 609s 812ms/step - loss: 0.0194 - acc: 0.9931 - val_loss: 0.0039 - val_acc: 0.9992

Epoch 00001: val_loss improved from inf to 0.00388, saving model to resnet50_blur_weights_01.h5
Epoch 2/20
750/750 [==============================] - 595s 794ms/step - loss: 0.0027 - acc: 0.9991 - val_loss: 0.0050 - val_acc: 0.9989

Epoch 00002: val_loss did not improve from 0.00388
Epoch 3/20
750/750 [==============================] - 578s 771ms/step - loss: 0.0065 - acc: 0.9984 - val_loss: 0.0114 - val_acc: 0.9945

Epoch 00003: val_loss did not improve from 0.00388
Epoch 4/20
750/750 [==============================] - 577s 769ms/step - loss: 0.0045 - acc: 0.9988 - val_loss: 0.0390 - val_acc: 0.9879

Epoch 00004: val_loss did not improve from 0.00388
Epoch 5/20
750/750 [==============================] - 584s 779ms/step - loss: 0.0023 - acc: 0.9993 - val_loss: 0.0023 - val_acc: 0.9997

Epoch 00005: val_loss improved from 0.00388 to 0.00226, saving mod